In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from numpy import random
import pandas as pd
import os
from dl_snpnet import fullNetwork
from efficient_snpnet efficientNetwork
from dl_utils import makeSNPSets, make_all_train_data, create_Y, get_train_data_registration_codes

In [6]:
torch.cuda.empty_cache()
import gc
gc.collect()
#device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
#net = fullNetwork(device, latent_size_factor = 2000, SnpSets = makeSNPSets(), preload_cache = False)
net = efficientNetwork(device, snpsets = makeSNPSets())
net = net.to(device)

Creating subnetwork number: 0/50
Creating subnetwork number: 1/50
Creating subnetwork number: 2/50
Creating subnetwork number: 3/50
Creating subnetwork number: 4/50
Creating subnetwork number: 5/50
Creating subnetwork number: 6/50
Creating subnetwork number: 7/50
Creating subnetwork number: 8/50
Creating subnetwork number: 9/50
Creating subnetwork number: 10/50
Creating subnetwork number: 11/50
Creating subnetwork number: 12/50
Creating subnetwork number: 13/50
Creating subnetwork number: 14/50
Creating subnetwork number: 15/50
Creating subnetwork number: 16/50
Creating subnetwork number: 17/50
Creating subnetwork number: 18/50
Creating subnetwork number: 19/50
Creating subnetwork number: 20/50
Creating subnetwork number: 21/50
Creating subnetwork number: 22/50
Creating subnetwork number: 23/50
Creating subnetwork number: 24/50
Creating subnetwork number: 25/50
Creating subnetwork number: 26/50
Creating subnetwork number: 27/50
Creating subnetwork number: 28/50
Creating subnetwork numb

In [8]:
Y_all = create_Y()
## match integer index in phenotype (different order than genetics binaries) to 10K RegistrationCodes
Y_all_ids = pd.Series(list(Y_all.index)).to_dict()
Y_all_ids = {v:k for k, v in Y_all_ids.items()}
Y_all = torch.from_numpy(np.array(Y_all)).to(device)

In [4]:
##match integer index in genetics cached binaries to 10K RegistrationCodes (for phenotype indexing)
test_size = 20
allIds = os.listdir("/net/mraid20/export/jasmine/zach/dl/person_cache/")
##randomly sample genetics indices to generate test data
test_ids_integer = np.random.choice(list(allIds.keys()), size = test_size) ##random sample of not trained_ids from genetics
test_ids_integer = list(filter(lambda id: Y_all_ids.get(allIds[id]) is not None, test_ids_integer))
##match the integer indices from genetics binaries to 10K ids
test_ids_tenK =  list(map(lambda intCode: allIds.get(intCode), test_ids_integer))
all_ids_not_test = list(set(allIds.keys()) - set(test_ids_integer)) ##Ids not used in training
##match the 10K codes we got above to ingeger indexes in the phenotype data
Y_ids_test = list(map(lambda test_id_tenK: Y_all_ids.get(test_id_tenK), test_ids_tenK))

In [9]:
train_sample_size = 5
criterion = nn.MSELoss()
optimizer = optim.Adam(params = net.parameters(), lr = 0.01)
numEpochs = 100
for epoch in range(numEpochs):
    ##Random sample to generate train and test data for each epoch
    ##only train on ids that map to 10K codes we have phenotype data for
    train_ids_integer_genetics = list(filter(lambda tenKid: Y_all_ids.get(allIds[tenKid]) is not None, np.random.choice(all_ids_not_test, size = train_sample_size)))
    ##map corresponding 10K ids to the integer indexes passed to the network
    train_ids_tenK = list(filter(lambda tenKid: Y_all_ids.get(tenKid) is not None, list(map(lambda intCode: allIds.get(intCode), train_ids_integer_genetics))))
    Y_ids = list(map(lambda train_id_tenK: Y_all_ids.get(train_id_tenK), train_ids_tenK))
    train_losses = []
    test_losses = []
    optimizer.zero_grad()
    outputs = net(train_ids_integer_genetics)
    ##don't backward pass NA otputs
    ignoreNAsIds_train = np.logical_and(~torch.isnan(outputs.cpu().detach()), ~torch.isnan(Y_all[[Y_ids]].cpu()))
    train_loss = criterion(outputs[ignoreNAsIds_train], Y_all[Y_ids][ignoreNAsIds_train])
    train_losses.append(train_loss)
    train_loss.backward()
    optimizer.step()
    if epoch %20 == 0:
        outputs_test = net(test_ids_integer)
        ignoreNAsIds_test = np.logical_and(~torch.isnan(outputs_test.cpu().detach()), ~torch.isnan(Y_all[[Y_ids_test]].cpu()))
        test_loss = criterion(outputs_test[ignoreNAsIds_test], Y_all[Y_ids_test][ignoreNAsIds_test])
        test_losses.append(test_loss)
        print("Test loss", test_loss.item())
        print("Epoch: ", epoch)
    print("Train loss: ", train_loss.item())

/home/zacharyl/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/IndexingUtils.h:27.)
  """Entry point for launching an IPython kernel.
/home/zacharyl/.local/lib/python3.7/site-packages/torch/autograd/__init__.py:175: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/IndexingUtils.h:27.)
  allow_unreachable=True, accumulate_grad=True)  # Calls into the C++ engine to run the backward pass


KeyboardInterrupt: 